In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from numpy import random
from dateutil.parser import parse
from re import search, escape, sub, findall
import re
import numpy as np
from os import mkdir, getcwd, path
from time import time
from glob import glob
from IPython.display import Audio
from datetime import datetime, timedelta
import pickle

In [2]:
# opening google chrome
driver = webdriver.Edge(r'C:\Users\user\Downloads\Softwares\edgedriver_win64\msedgedriver.exe')

In [3]:
# f1 = 300.0
# f2 = 300.0
# rate = 8000.0
# L = 3
# times = np.linspace(0,L,int(rate*L))
# signal = np.sin(2*np.pi*f1*times) + np.sin(2*np.pi*f2*times)
# i=0
# while i < len(signal):
#     signal[i:i+1000] = np.array([0]*1000)
#     i+=5000

In [4]:
def create_dir(name):
    print("Creating folder: %s"%path.basename(name))
    try:  
        mkdir(name)
    except OSError as error:  
        print(error)   

In [44]:
MIN_SLEEP_TIME = 10
MAX_SLEEP_TIME = 15

PLATFORM_NAME = "Wink-Wikis"
start_date = '2012-01-01'
end_date = '2020-07-01'
DATA_PATH  = getcwd() + '\\' + PLATFORM_NAME
create_dir(DATA_PATH)
DATA_PATH += "\\"

Creating folder: Wink-Wikis


In [45]:
def random_sleep():
    sleep_time = random.choice([i for i in range(MIN_SLEEP_TIME,MAX_SLEEP_TIME)])
    print("- Sleeping for %d seconds -"%sleep_time)
    sleep(sleep_time)

In [46]:
# function to load the page
def load_page_url(driver, url):
    print("- Loading page using driver -")
    try:
        driver.get(url) # loading page
    except:
        print("- Loading page failed -")
    random_sleep()

In [47]:
# funtion to get the page source code
def get_page_source(driver):
    print("- Getting Page Source -")
    soup = BeautifulSoup(driver.page_source, "lxml") # using Beautiful Soup Module to parse and process the source 
    return soup

In [48]:
# function to convert string date to date format
def string_to_date(date):
    return datetime.strptime(date, '%Y-%m-%d').date()

In [49]:
def clean_text(text):
    text = sub('\n','', text) # to remove newlines
    text = sub(r'[ ]+',' ', text) # to remove extra space
    text = text.strip() # to remove space at start and end
    return text

In [50]:
def identify_n_issues(issuses):
    issuses = sub(',','', issuses)
    return int(findall(r'[\d]+',issuses)[0])

In [51]:
# function to load issues between specified date
def load_github_issues(start_date, end_date):
    
    URL = "https://github.com/search?p=1&q=%28%22wink+api%22+OR+%22wink+developer%22+OR+%22wink+hub%22%29+updated%3A" + start_date + ".." + end_date + "&type=Wikis"
    load_page_url(driver, URL)
    pg_source = get_page_source(driver)
    n_issues = pg_source.find(name='div', attrs={'class':'d-flex flex-column flex-md-row flex-justify-between border-bottom pb-3 position-relative'})
    n_issues = identify_n_issues(clean_text(n_issues.find('h3').text))
    issues_per_page = 10
    pages_to_load = int(n_issues/issues_per_page)
    if n_issues/issues_per_page > 0 :
        pages_to_load+=1
        
    return n_issues, pages_to_load

In [52]:
# function to decrease end date to lessen the number of results
def decrease_end_date(start_date, end_date, smart_decrease = False, cur_result_cnt=0, max_result_cnt = 1000):
    
    print("- Decreasing end date (smart decrease: %s) -"%str(smart_decrease))
    start_date = string_to_date(start_date)
    end_date = string_to_date(end_date)
    
    # getting current difference
    dif_days = end_date - start_date
    dif_days = dif_days.days 
    
    print("- Current difference between %s and %s: %d days -"%(str(start_date), str(end_date), dif_days))
    if (smart_decrease) and (cur_result_cnt > 0):
        end_date = start_date + timedelta(days= int(max_result_cnt/(cur_result_cnt/dif_days)))
    else:
        end_date = start_date + timedelta(days=int(dif_days/2))
    
    # getting new difference
    dif_days = end_date - start_date
    dif_days = dif_days.days 
    print("- Updated difference between %s and %s: %d days -"%(str(start_date), str(end_date), dif_days))
        
    return str(end_date)

In [53]:
# function to extract issues from a page
def extract_page_issues(page_source):
    issues = page_source.find_all(name='div', attrs={'class':'hx_hit-wiki py-4 border-top'})
    page_data = []
    for issue in issues:
        user_name = issue.find(name='a', attrs={'class':'muted-link text-small text-bold'}).text
        open_date = issue.find(name='relative-time')['title']
        wiki_id = issue.find(name='div', attrs={'class':'f4 text-normal'}).find('a')['data-hydro-click-hmac']

        page_data.append([user_name, open_date, wiki_id])
    return page_data

In [54]:
# function to save issues from a page
def save_page_issues(page_issues, start_date, end_date, page_no, type_='txt'):
    print('- Writing %d issues from page_no: %d between dates %s and %s -'%(len(page_issues),page_no,start_date, end_date))
    # write data in a file. 
    if type_ == "txt":
        file_name = sub("-","",start_date) + '-' + sub("-","", end_date) + '-' + str(page_no) + '.txt'
        file_ = open(DATA_PATH + file_name,"w") 
        file_.writelines(str(page_issues)) 
        file_.close() #to change file access modes 
        return 

    file_name = sub("-","",start_date) + '-' + sub("-","", end_date) + '-' + str(page_no) + '.pickle'
    with open(DATA_PATH+ file_name, 'wb') as filehandle:
        # store the data as binary data stream
        pickle.dump(page_issues, filehandle)
    return 

In [55]:
# function to update page no of specified url
def update_url_page_no(URL, page_no):
    return sub(r'&p=[\d]+', '&p='+str(page_no), URL)  

In [56]:
GLOBAL_CUR_PAGE_NO = 1
GLOBAL_START_DATE = start_date
GLOBAL_END_DATE = end_date

In [57]:
def main(start_date, end_date, GLOBAL_CUR_PAGE_NO=1):
    
    data = []
    while start_date < end_date: # running this loop to get all issuses from main start and end dates
        print(" ------------------------------------------------------------------------------------ ")
        n_issues, pages_to_load = load_github_issues(start_date=start_date, end_date=end_date)
        print("- Found %d issues and %d pages to load from %s and %s"%(n_issues, pages_to_load, start_date, end_date))
        approximated_end_date = end_date
        
        
        while n_issues > 1000: # running this loop to approximate scrappable end date
            approximated_end_date = decrease_end_date(start_date=start_date, 
                                                      end_date=approximated_end_date, 
                                                      smart_decrease=True, 
                                                      cur_result_cnt=n_issues, 
                                                      max_result_cnt=1000)
            
            n_issues, pages_to_load = load_github_issues(start_date=start_date, end_date=approximated_end_date)
            print("- Found %d issues and %d pages to load from %s and %s"%(n_issues, pages_to_load, start_date, approximated_end_date))
            random_sleep()
         
        # scrapping over pages 
        cur_page_no = 1
        
        
        if GLOBAL_CUR_PAGE_NO != 1: 
            cur_page_no = GLOBAL_CUR_PAGE_NO
            GLOBAL_CUR_PAGE_NO = 1
            
            print("- Setting cur_page_no to %d -"%cur_page_no)
            
            
        cur_page_url = driver.current_url
        while cur_page_no <= pages_to_load:
            print("- Getting issues from page %d -"%cur_page_no)
            
            # loading next page
            if cur_page_no != 1:
                cur_page_url = update_url_page_no(URL=cur_page_url, page_no=cur_page_no)
                load_page_url(driver=driver, url=cur_page_url)
            
            # extracting issues
            pg_source = get_page_source(driver)
            page_issues = extract_page_issues(pg_source)
            # saving output of every page
            if (page_issues is not None and len(page_issues) > 0):
                save_page_issues(page_issues=page_issues,
                                 start_date=start_date, 
                                 end_date=approximated_end_date, 
                                 page_no=cur_page_no, 
                                 type_='pickle')
            cur_page_no += 1 # updating page number
        
        # updating start date
        start_date = approximated_end_date

In [58]:
if GLOBAL_START_DATE != start_date:
    print("1")
    main(start_date=GLOBAL_START_DATE, end_date=GLOBAL_END_DATE, GLOBAL_CUR_PAGE_NO=GLOBAL_CUR_PAGE_NO)
else:
    print("2")
    main(start_date, end_date)

2
 ------------------------------------------------------------------------------------ 
- Loading page using driver -
- Sleeping for 14 seconds -
- Getting Page Source -
- Found 53 issues and 6 pages to load from 2012-01-01 and 2020-07-01
- Getting issues from page 1 -
- Getting Page Source -
- Writing 10 issues from page_no: 1 between dates 2012-01-01 and 2020-07-01 -
- Getting issues from page 2 -
- Loading page using driver -
- Sleeping for 10 seconds -
- Getting Page Source -
- Writing 10 issues from page_no: 2 between dates 2012-01-01 and 2020-07-01 -
- Getting issues from page 3 -
- Loading page using driver -
- Sleeping for 14 seconds -
- Getting Page Source -
- Writing 10 issues from page_no: 3 between dates 2012-01-01 and 2020-07-01 -
- Getting issues from page 4 -
- Loading page using driver -
- Sleeping for 12 seconds -
- Getting Page Source -
- Writing 10 issues from page_no: 4 between dates 2012-01-01 and 2020-07-01 -
- Getting issues from page 5 -
- Loading page using dr